In [ ]:
from skimpy import skim
from psycop_coercion.utils.synth_data import create_synth_prediction_times

df_prediction_times = create_synth_prediction_times()

skim(df_prediction_times)
df_prediction_times.sort_values(by=["entity_id"])

In [ ]:
from psycop_coercion.utils.synth_data import create_synth_feature_binary

feature_binary = create_synth_feature_binary()

skim(feature_binary)
feature_binary.sort_values(by=["entity_id"])

In [ ]:
from timeseriesflattener.feature_spec_objects import PredictorSpec
from timeseriesflattener.resolve_multiple_functions import boolean
import numpy as np

pred_spec = PredictorSpec(
    values_df=feature_binary,
    lookbehind_days=365,
    fallback=np.nan,
    resolve_multiple_fn=boolean,
    feature_name="value",
)

In [ ]:
from psycop_coercion.utils.synth_data import create_synth_feature_float

feature_float = create_synth_feature_float()

skim(feature_float)
feature_float.sort_values(by=["entity_id"])

In [ ]:
from timeseriesflattener.feature_spec_objects import PredictorGroupSpec
from timeseriesflattener.resolve_multiple_functions import mean, minimum, maximum

pred_group_spec = PredictorGroupSpec(
    values_df=feature_float,
    lookbehind_days=[365],
    fallback=[np.nan],
    resolve_multiple_fn=[mean, minimum, maximum,],
    feature_name="value",
).create_combinations()

In [ ]:
from psycop_coercion.utils.synth_data import create_synth_feature_text

feature_text = create_synth_feature_text()

skim(feature_text)
feature_text.sort_values(by=["entity_id"])

In [ ]:
from timeseriesflattener.feature_spec_objects import TextPredictorSpec
from timeseriesflattener.resolve_multiple_functions import concatenate

text_spec = TextPredictorSpec(
    values_df=feature_float,
    lookbehind_days=365,
    fallback=np.nan,
    resolve_multiple_fn=concatenate,
    feature_name="value",
)

In [ ]:
from timeseriesflattener.flattened_dataset import TimeseriesFlattener

ts_flattener = TimeseriesFlattener(
    prediction_times_df=df_prediction_times,
    entity_id_col_name="entity_id",
    timestamp_col_name="timestamp",
    n_workers=4,
    drop_pred_times_with_insufficient_look_distance=True,
)

In [ ]:
specs = pred_group_spec + [pred_spec] + [text_spec]

ts_flattener.add_spec(specs)

flattened_df = ts_flattener.get_df()

In [ ]:
skim(flattened_df)

flattened_df.sort_values(by=["entity_id"])